<a href="https://colab.research.google.com/github/yashc73080/CS462-Deep-Learning/blob/main/HW2/features_unsupervised_and_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import time
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# 1. Auto-Encoders

In [3]:
mnist_tfm = transforms.Compose([transforms.ToTensor()])
mnist_train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=mnist_tfm)
mnist_test_ds = datasets.MNIST(root="./data", train=False, download=True, transform=mnist_tfm)

mnist_train_loader = DataLoader(mnist_train_ds, batch_size=128, shuffle=True, num_workers=2)
mnist_test_loader = DataLoader(mnist_test_ds, batch_size=256, shuffle=False, num_workers=2)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.95MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.24MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 11.5MB/s]


In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_size=784, enc_hidden=500, bottleneck_k=200, device='cpu'):
    '''
    Encoder module with architecture: Input (784) -> Hidden -> Bottleneck k
    '''
    super().__init__()

    layers = []
    layers.append(nn.Linear(input_size, enc_hidden))
    layers.append(nn.ReLU())
    layers.append(nn.Linear(enc_hidden, bottleneck_k))

    self.encoder = nn.Sequential(*layers)

  def forward(self, x):
    return self.encoder(x)


class Decoder(nn.Module):
  def __init__(self, bottleneck_k=200, dec_hidden=500, output_size=784, device='cpu'):
    '''
    Decoder module with architecture: Bottleneck k -> Hidden -> Output (784)
    '''
    super().__init__()

    layers = []
    layers.append(nn.Linear(bottleneck_k, dec_hidden))
    layers.append(nn.ReLU())
    layers.append(nn.Linear(dec_hidden, output_size))

    self.decoder = nn.Sequential(*layers)

  def forward(self, x):
    return self.decoder(x)


class AutoEncoder(nn.Module):
  def __init__(self, in_out_size=784, enc_hidden=500, dec_hidden=500, bottleneck_k=200, device='cpu'):
    '''
    AutoEncoder module with architecture: Input (784) -> Hidden -> Bottleneck k -> Hidden -> Output (784)
    '''
    super().__init__()
    self.encoder = Encoder(input_size=in_out_size, enc_hidden=enc_hidden, bottleneck_k=bottleneck_k, device=device)
    self.decoder = Decoder(bottleneck_k=bottleneck_k, dec_hidden=dec_hidden, output_size=in_out_size, device=device)

    self.to(device)

  def forward(self, x):
    return self.decoder(self.encoder(x))


def train_autoencoder(model):
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  reconstruction_loss = nn.MSELoss()

  model.train()
